<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition_STT/blob/main/STT_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 19.4 MB/s eta 0:00:00


In [3]:
import speech_recognition as sr
import pandas as pd
from tqdm import tqdm
import re

# DATA

In [4]:
data_emotion = pd.read_csv('data_emotion.csv')
data_main = pd.read_csv('data_main.csv')

In [5]:
data_emotion

,Emotion,Path
0,sad,/content/drive/MyDrive/project/data_aihub/5/5e...
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
2,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
3,sad,/content/drive/MyDrive/project/data_aihub/5/5f...
4,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
...,...,...
12674,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12675,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12676,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12677,happy,/content/drive/MyDrive/project/data_aihub/5/5e...


In [6]:
data_main

,Path
0,/content/drive/MyDrive/project/aihub_data_main...
1,/content/drive/MyDrive/project/aihub_data_main...
2,/content/drive/MyDrive/project/aihub_data_main...
3,/content/drive/MyDrive/project/aihub_data_main...
4,/content/drive/MyDrive/project/aihub_data_main...
...,...
409,/content/drive/MyDrive/project/aihub_data_main...
410,/content/drive/MyDrive/project/aihub_data_main...
411,/content/drive/MyDrive/project/aihub_data_main...
412,/content/drive/MyDrive/project/aihub_data_main...


# STT

## emotion

In [7]:
recognizer = sr.Recognizer()

In [8]:
def speech_to_text(file_path):
  try:
    with sr.AudioFile(file_path) as source:
      audio = recognizer.record(source)
      text = recognizer.recognize_google(audio, language = "ko-KR")
      return text
  except sr.UnknownValueError:
    return "음성을 인식할 수 없음"
  except sr.RequestError as e:
    return f"STT API 오류: {e}"

In [9]:
data_emotion['Text'] = [speech_to_text(path) for path in tqdm(data_emotion['Path'])]

100%|██████████| 12679/12679 [3:11:02<00:00,  1.11it/s]


In [10]:
print(data_emotion[['Path', 'Text']])

                                                    Path  \
0      /content/drive/MyDrive/project/data_aihub/5/5e...   
1      /content/drive/MyDrive/project/data_aihub/4/5e...   
2      /content/drive/MyDrive/project/data_aihub/5_2/...   
3      /content/drive/MyDrive/project/data_aihub/5/5f...   
4      /content/drive/MyDrive/project/data_aihub/5_2/...   
...                                                  ...   
12674  /content/drive/MyDrive/project/data_aihub/5_2/...   
12675  /content/drive/MyDrive/project/data_aihub/5_2/...   
12676  /content/drive/MyDrive/project/data_aihub/5_2/...   
12677  /content/drive/MyDrive/project/data_aihub/5/5e...   
12678  /content/drive/MyDrive/project/data_aihub/5_2/...   

                                 Text  
0                        고마워 지금 하고 올게  
1                        음성을 인식할 수 없음  
2                            주식을 시작했어  
3      어 그렇게 심한 건 아니고 허리 다쳐서 병원에 입원 중  
4           음악을 들으면 우울한 내 마음이 좀 가라앉으며  
...                               ...  

In [11]:
data_emotion.to_csv('emotion_text.csv', index = False, encoding = 'utf-8-sig')

## main

In [15]:
recognizer = sr.Recognizer()

In [ ]:
def speech_to_text(file_path):
    try:
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [ ]:
def split_into_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return sentences

In [ ]:
data_list = []
for path in tqdm(data_main['Path']):
    text = speech_to_text(path)
    sentences = split_into_sentences(text)

    for sentence in sentences:
        data_list.append({'Path': path, 'Text': sentence})

In [ ]:
main_text = pd.DataFrame(data_list)

In [ ]:
print(main_text)

In [ ]:
main_text.to_csv('main_text.csv', index = False, encoding = 'utf-8-sig')